In [57]:
from bs4 import BeautifulSoup as bs
import urllib.request, sys, time
import requests
import pandas as pd
from io import StringIO
from html.parser import HTMLParser
import re

In [58]:
class HuffpostNewsBodyScrapper:
    def __init__(self, url):
        self.body_wrapper = None
        article = requests.get(url);
        self.soup = bs(article.content, "html.parser")

        self.body = self.get_body()

    def get_body(self):
        self.body_wrapper = self.soup.find("section", attrs={'class', 'entry__content-list'})
        text = []
        paragraphs = self.body_wrapper.select("div.primary-cli.cli.cli-text")

        for para in paragraphs:
            para_content = para.find('p')
            if para_content is not None:
                    # get text except the tag
                childs = para_content.find_all(recursive=False)
                # print(childs)
                # if have child
                if len(childs) > 0:
                    for child in childs:
                        child_text = strip_tags(str(child))
                        text.append(child_text)
                else:
                    para_text = strip_tags(str(para_content))
                    # print(para_text)
                    text.append(para_text)
        return re.sub('[”’"“]', " ", (''.join(text))).replace(",", ".")


In [59]:
# stripping HTML TAGs
class TAGStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    stripper = TAGStripper()
    stripper.feed(html)
    return stripper.get_data()


In [60]:
def getHoffPostNews(URL,start, pages, category, file):
    for pageno in range(start, start + pages):
        print('webscrapping page :', pageno)
        request_URL = URL + str(pageno)
        print(request_URL)
        try:
            webpage = requests.get(request_URL)
        except Exception as e:
            error_type, error_obj, error_info = sys.exc_info()
            print('Error for link:', request_URL)
            print(error_type, 'Line:', error_info.tb_lineon)
            continue

        time.sleep(5)
        soup = bs(webpage.text, 'html.parser')
        article_cards = soup.select('div.card.card--standard')
        # print(len(article_cards))

        for card in article_cards:
            link = card.find('a', attrs={'class': 'card__headline'})
            href = link['href']
            title = link.find('h3').text
            body = HuffpostNewsBodyScrapper(href).get_body()
            # author = card.select_one('div.card__byline__author__name-title').text
            single_author = card.find('div', attrs={'class': 'card__byline__author'}).find('div', attrs={'class': 'card__byline__author__name-title'})
            author = ""
            if single_author is None:
                mutiple_author = card.find('div', attrs={'class': 'card__byline__author--external'}).text
                author = mutiple_author
            else:
                author = single_author.text

            # lower_frame = [href, title, author, body, category]
            # frame.append(lower_frame)
            file.write(
                body + ","
                + category
                + "\n"
            )

In [67]:
filename="HuffPostNews_Train.csv"
news = [
    # {
    #     "category": "Business News",
    #     "page": 5,
    #     "link": "https://www.huffpost.com/impact/business?page=",
    # },
    # {
    #     "category": "Entertainment News",
    #     "page": 30,
    #     "link": "https://www.huffpost.com/entertainment/?page=",
    # },
    {
        "category": "Politics News",
        "page": 35,
        "link": "https://www.huffpost.com/news/politics?page=",
    },
    # {
    #     "category": "Health News",
    #     "page": 100,
    #     "link": "https://www.huffpost.com/section/health?page=",
    # }
]

# creating csv file
f=open(filename, "w", encoding = 'utf-8')
headers="Text,Category\n"
f.write(headers)

for new in news:
    print(f"Getting {new['category']}")
    getHoffPostNews(new['link'], 1, new['page'], new['category'], f)
    # time.sleep(300)
f.close()

print("finish")

Getting Politics News
webscrapping page : 1
https://www.huffpost.com/news/politics?page=1
webscrapping page : 2
https://www.huffpost.com/news/politics?page=2
webscrapping page : 3
https://www.huffpost.com/news/politics?page=3
webscrapping page : 4
https://www.huffpost.com/news/politics?page=4
webscrapping page : 5
https://www.huffpost.com/news/politics?page=5
webscrapping page : 6
https://www.huffpost.com/news/politics?page=6
webscrapping page : 7
https://www.huffpost.com/news/politics?page=7
webscrapping page : 8
https://www.huffpost.com/news/politics?page=8


KeyboardInterrupt: 

In [85]:
category = "Business News"
text = open("text.txt", "r")
body = text.read()
body = body.replace("\n", "").replace(",", ".")
body = re.sub('[”’"“]', " ", body).strip()
# print(body)
text.close()
csv = open("SCMP_Val.csv", "a")
csvline = body+","+category+"\n"
print(csvline)
csv.write(csvline)
csv.close()



The highly transmissible XBB could become the next dominant strain of Covid-19 in Hong Kong. but the population s level of hybrid immunity should stave off severe illnesses. health experts have said.The city has recorded 140 XBB infections since an overseas traveller became the first known carrier on October 10. but the number still pales beside other cases involving dominant strains in the city. such as BA.4 or BA.5. They accounted for 95.1 per cent of local cases over the past two weeks. compared with less than 0.1 per cent involving XBB. XBB could be the next dominant strain in Hong Kong.  said Professor Ivan Hung Fan-ngai. a top infectious disease expert at the University of Hong Kong.  Nevertheless. XBB is a recombinant of BA 2.75 and 2.10.1. Most people in Hong Kong have been infected with Omicron BA.2 or BA.5 before. plus [they have] a very high vaccination rate. Therefore. [they have] very high hybrid immunity. The immunity. coupled with a lack of evidence showing XBB was more 